## Decorators

Un decorator in Python è una funzione (più in generale si può dire un **oggetto chiamabile**) che viene usato per modificare (decorare) il comportamento di una funzione o di una classe (e dei suoi oggetti).

Il decoratore prende in input la funzione (o la classe) e restituisce la funzione (o la classe) modificata

Abbiamo già utilizzato questa struttura (senza averla chiamata col nome appropriato) quando abbiamo usato @property, che un esempio notevole di decoratore

Ora vediamo la nozione in modo più diretto

### Ricordiamo che le funzioni sono first-class object

In [9]:
def Fibonacci(n):
    f0 = 0
    f1 = 1
    for _ in range(n-1):
      f0, f1 = f1, f0+f1
    return f1

In [ ]:
Fibonacci(100)

In [ ]:
fibo = Fibonacci
fibo(10)

In [ ]:
def tabulate(sequence, f):
    return [f(x) for x in sequence]

In [ ]:
import math
tabulate([x*0.01 for x in range(157)], math.sin)

In [ ]:
def maketab(start, step, npoints):
    sequence = [start+i*step for i in range(npoints)]
    def tabulate(f):
        return [f(x) for x in sequence]
    return tabulate

In [ ]:
tabulate = maketab(0,0.01,157)

In [ ]:
tabulate(math.cos)

In [ ]:
def makePoly(*coefficients):
    def eval(x):
        value = 0.0
        for c in reversed(coefficients):
            value = value*x + c
        return value
    return eval

In [ ]:
p = makePoly(1,2,-1)

In [ ]:
for i in range(5):
    print(p(i))

### I decoratori mettono insieme le due "costruzioni": funzione come parametro e funzione come valore di ritorno, con una **sintassi specifica**

In [ ]:
def decoratore(f):
    def decorata(x):
        print('x\t f(x)')
        f(x)
    return decorata

In [ ]:
def g(x):
    print(x,'\t',math.sin(x))

In [ ]:
g(0.2)

In [ ]:
g = decoratore(g)

In [ ]:
gdec(0.2)

In [ ]:
@decoratore
def g(x):
    print(x,'\t',math.sin(x))

In [ ]:
g(0.2)

In [ ]:
def decoratore(f):
    def decorata(*x):
        print('x\t f(x)')
        for z in x:
            f(z)
    return decorata

In [ ]:
g(0.01,0.02,0.03)

In [ ]:
g([i/10 for i in range(10)])

### Tipici casi d'uso dei decoratori

La decorazione può utilmente "occuparsi" (in particolare, per la chiarezza del codice) dei casi particolari di input che devono essere controllati da un algoritmo.

In [ ]:
def roots2(a,b,c):
    delta = math.sqrt(b**2-4*a*c)
    x1 = (-b-delta)/(2*a)
    x2 = (-b+delta)/(2*a)
    return x1,x2

In [ ]:
roots2(0,-2,1)

In [ ]:
def special_cases(f):
    '''Per semplicità, tratta solo il caso in cui il coefficiente del termine quadratico è nullo'''
    def checker(a,b,c):
        if a==0:
            return -c/b
        else:
            return f(a,b,c)
    return checker

In [ ]:
@special_cases
def roots2(a,b,c):
    delta = math.sqrt(b**2-4*a*c)
    x1 = (-b-delta)/(2*a)
    x2 = (-b+delta)/(2*a)
    return x1,x2

In [ ]:
roots2(0,-2,1)

Un secondo caso di utilizzo è la raccolta di statistiche sull'uso di una funzione (tempo di esecuzione, numero di volte che è stata chiamata, ...)

In [1]:
from time import time

In [5]:
time()

1543412393.8636892

In [24]:
n = 100

In [14]:
start_time = int(round(time() * 1000)) #tempo in millisecondi (trascorso a partire dal 1/1/1970
Fibonacci(n)
stop_time = int(round(time() * 1000))
elapsed_time = stop_time-start_time
print("Tempo di esecuzione di Fibonacci({}): {}ms".format(n, elapsed_time))

Tempo di esecuzione di Fibonacci(1000): 0ms


In [25]:
def my_timer(f):
    def timedfun(n):
        start_time = int(round(time() * 1000))
        res = f(n)
        stop_time = int(round(time() * 1000))
        timedfun.time = stop_time-start_time
        return res
    return timedfun

@my_timer
def Fibonacci(n):
    f0 = 0
    f1 = 1
    for _ in range(n-1):
      f0, f1 = f1, f0+f1
    return f1

res = Fibonacci(n)
print("Tempo di esecuzione di Fibonacci({}): {}ms".format(n, Fibonacci.time))
print(res)

Tempo di esecuzione di Fibonacci(100): 0ms
354224848179261915075


In [ ]:
Fibonacci(n)
print("Tempo di esecuzione di Fibonacci({}): {}ms".format(n, Fibonacci.time))

In [26]:
@my_timer
def insSort(A):
    n = len(A)
    for i in range(1,n):
        temp = A[i]
        j = i-1
        while j>=0 and A[j]>temp:
            A[j+1] = A[j]
            j = j-1
        A[j+1] = temp

In [27]:
from random import randint

In [35]:
n = 20000
A = [randint(1,100) for _ in range(n)]
#A

In [46]:
insSort(A)
print("Tempo di esecuzione di insSort(A): {}ms".format(insSort.time))

1
Tempo di esecuzione di insSort(A): 13ms


In [31]:
A

[15, 18, 21, 21, 29, 34, 39, 43, 63, 84]

In [59]:
def general_timer(f):
    def timedfun(*args, **kw):
        start_time = int(round(time() * 1000))
        res = f(*args, **kw)
        stop_time = int(round(time() * 1000))
        timedfun.time = stop_time-start_time
        return res
    return timedfun

In [39]:
@general_timer
def insSort(A):
    n = len(A)
    for i in range(1,n):
        temp = A[i]
        j = i-1
        while j>=0 and A[j]>temp:
            A[j+1] = A[j]
            j = j-1
        A[j+1] = temp

In [42]:
insSort(A)
print("Tempo di esecuzione di insSort(A): {}ms".format(insSort.time))

1
Tempo di esecuzione di insSort(A): 3ms


Occhio ai problemi con le funzioni ricorsive... (a cominciare dal valore di n)

In [73]:
n = 25

In [86]:
def Fibonacci_rec(n):
    if n == 0 or n == 1:
        return n
    return Fibonacci_rec(n-2)+Fibonacci_rec(n-1)

In [80]:
start_time = int(round(time() * 1000)) #tempo in millisecondi (trascorso a partire dal 1/1/1970)
Fibonacci_rec(n)
stop_time = int(round(time() * 1000))
elapsed_time = stop_time-start_time
print("Tempo di esecuzione: {}ms".format(elapsed_time))

Tempo di esecuzione: 26ms


In [81]:
@general_timer
def Fibonacci_rec(n):
    if n == 0 or n == 1:
        return n
    return Fibonacci_rec(n-2)+Fibonacci_rec(n-1)

In [85]:
Fibonacci_rec(n)
print(Fibonacci_rec.time)

220


Una possibile soluzione (poco "elegante" ...).

In [87]:
@general_timer
def Fibonacci_r(n):
    if n == 0 or n == 1:
        return n
    return Fibonacci_rec(n-2)+Fibonacci_rec(n-1)

In [88]:
Fibonacci_r(n)
print(Fibonacci_r.time)

35


Anziché il tempo, si potrebbe voler misurare il numero di chiamate di una funzione

In [90]:
def callcnt(f):
    def counter(*args, **kw):
        counter.calls += 1
        return f(*args, **kw)
    counter.calls = 0
    return counter

Questa volta non ci sono problemi con la ricorsione

In [91]:
@callcnt
def Fibonacci_rec(n):
    if n == 1 or n == 2:
        return 1
    return Fibonacci_rec(n-2)+Fibonacci_rec(n-1)

In [94]:
n = 5
Fibonacci_rec.calls = 0
v = Fibonacci_rec(n)
print("Il numero C(n) di chiamate ricorsive (con n = {}) è {}".format(n,Fibonacci_rec.calls))
print("Il valore di f({}) è {}".format(n,v))
print("Controllo, C(n) == 2*f(n)-1: {}".format(Fibonacci_rec.calls==2*v-1))

Il numero C(n) di chiamate ricorsive (con n = 5) è 9
Il valore di f(5) è 5
Controllo, C(n) == 2*f(n)-1: True


### Anche le classi possono essere utilizzate come decoratori

### Digressione. \__call\__: un interessante "metodo magico"

In [95]:
from random import random

In [113]:
random()

0.4726666929092227

In [99]:
class myrand:
    '''Un semplice generatore pseudo-casuale, da non usare per scopi crittografici...'''
    def __init__(self):
        self._a = 16807
        self._m = (1 << 31) - 1 #2^31 -1
        self._d = 1.0/self._m
        self._x = 1
        for i in range(10000):  # Per rendere i numeri "usati" (apparentemente) indipendenti dal seme iniziale
            self.__call__()
    
    def __call__(self):
        self._x = (self._a*self._x)%self._m
        return self._x*self._d

In [100]:
r = myrand()

In [101]:
for _ in range(10):
    print(r())

0.7403424972390488
0.9363510966935898
0.2528821291648234
0.18994487318673398
0.40348364943800663
0.34969610457760103
0.34242963574008534
0.21488788361423083
0.6206599043778422
0.43101287839515734


E' evidente che un tale generatore (del quale tutte le possibili istanze si comporterebbero alla stessa maniera) può essere realizzato in molti altri modi. Tuttavia l'esempio illustra il punto. Il metodo \__call\__ può essere chiamato col nome stesso della classe, il che rende l'uso di comprensione immediata. L'esempio illustra come sua possibile realizzare semplicemente una sorta di "funzione con stato", mediante classi e metodo \__call\__

Esempio: generazione di numeri distribuito con esponenziale negativa (di fondamentale utilità in esperimenti di simulazione)

In [114]:
from math import log
class negexp:
    '''Utilizza un qualsiasi generatore uniforme e lo utilizza
       per calcolare "negative exponential deviate". l si interpreta
       come frequenza degli eventi (numero di eventi nell'unità di tempo)'''
    def __init__(self, l):
        self._lambda = l

    def __call__(self, fn):
        '''fn è il generatore uniforme'''
        def decorated():
            r = fn()
            return -log(1.0-r)/self._lambda
        return decorated

In [149]:
@negexp(0.5)
def schedule():
    return random()

In [155]:
schedule()

0.017254466644020715

In [ ]:
negexp(2).__call__(random)()

In [162]:
t = 0.0
count = 0
while t<100:
    t += schedule()
    count += 1
print(count)

53


In [ ]:
@negexp(10)
def schedulefast():
    return random()

In [ ]:
t = 0.0
count = 0
while t<100:
    t += schedulefast()
    count += 1
print(count)

In [1]:
###### Progetto di un semplice simulatore con 1 coda e k serventi

from math import log
from random import random

import pdb

JOB_ARRIVAL = 0

class negexp:
    '''Utilizza un qualsiasi generatore uniforme e lo utilizza
       per calcolare "negative exponential deviate". l si interpreta
       come frequenza degli eventi (numero di eventi nell'unità di tempo)'''
    def __init__(self, l):
        self._lambda = l

    def __call__(self, fn):
        '''fn è il generatore uniforme'''
        def decorated():
            x = fn()
            return -log(1.0-x)/self._lambda
        return decorated

class globalTime:
    
    def __init__(self, start = 0):
        self.__time = start
        
    def advance(self, newTime):
        if newTime > self.__time:
            self.__time = newTime
        
    def __call__(self):
        return self.__time
    
    @property
    def time(self):
        return self.__time

class eventType:
    '''Introduce "costanti" per identificare il tipo di evento'''
    JOB_ARRIVAL = -1
    JOB_COMPLETION = 1
    
    @classmethod
    def __call__(cls, e):
        if e == cls.JOB_ARRIVAL:
            return cls.JOB_ARRIVAL
        else:
            return cls.JOB_COMPLETION
 
class event(int):
    '''Classe contenitore, dalla quale ereditano arrival e job. 
    1) arrival è la richiesta di un nuovo servizio (arrivo di un nuovo job) e,
    come tale, è caratterizzata dal solo attributo time (che viene usato anche
    come priorità per gestire l'ordine di accadimento degli eventi).
    2) job descrive più in generale un "oggetto" di tipo job. A stretto rigore,
    un job diviene anche un evento quando per esso è schedulato il tempo di fine
    (tramite la chiamata di inService). Se quindi si dovesse esegure un 
    confronto del tipo j<a, volto a determinare se il job j precede l'arrivo a,
    si avrebbe un errore se j non è già stato schedulato per il servizio (e dunque
    non se ne conosce il tempo di fine).
    La classe eredita da int e dunque gli eventi sono rappresentati come numeri interi.

    '''
    def __new__(cls, t):
        e = super().__new__(cls,t)
        return e
        
    def __lt__(self, other):
        return self.time < other.time
    
class arrival(event):
    '''Classe che rappresenta gli eventi di arrivo di richieste di servizio'''
    def __new__(cls, time):
        e = super().__new__(cls, eventType.JOB_ARRIVAL)
        e.__priority = time
        return e
    
    @property
    def time(self):
        return self.__priority

class job(event):
    '''Classe che definisce i job. Questi sono identificati semplicemente da 
    numeri interi (ereditano da int via event) con alcune proprietà utilizzate 
    per la gestione e per il calcolo delle statistiche:
    1) tempo di arrivo nel sistema
    2) tempo di inizio servizio
    3) tempo di fine servizio
    4) identificazione del server
    Si noto che il sistema è senza pre-emption (i job non vengono interrotti)
    '''
    
    __jobCnt = 1  # Contatore "automatico" per attribuire un id (int) ai job
    
    def __new__(cls, arrivalTime):
        j = super().__new__(cls, job.__jobCnt) # chiamata del costruttore della classe parent (int)
        j.__arrivalTime = arrivalTime          # tempo di arrivo del job
        job.__jobCnt += 1                      # incremento del contatore
        return j
    
    def __str__(self):
        j = super().__str__()
        return "job {}\n\tarrival time: {}\n\tserver: {}\n\tservice time: {}\n\tend time: {}".\
                format(j, self.arrivalTime, self.server.server, self.serviceTime, self.endTime)
        '''        
        return "job {}\n\tarrival time: {}\n"+\
               "\tserver: {}\n\tservice time: {}\n"+\
               "\tend time: {}".format(j, self.arrivalTime, self.server,\
                                self.serviceTime, self.endTime)
        '''
        
    @classmethod
    def resetCounter(cls):
        cls.__jobCnt = 1

    def inService(self, time, server, duration):
        '''Memorizza i tempi di inizio e fine servizio e il numero del servente'''
        self.__start = time
        self.__priority = time+duration
        self.__server = server
        
    @property
    def server(self):
        return self.__server
    
    @property
    def arrivalTime(self):
        return self.__arrivalTime
    
    @property
    def serviceTime(self):
        try:
            t = self.__start
        except:
            print("Il job è ancora in coda")
            return None
        return t
    
    @property
    def endTime(self):
        try:
            t = self.__priority
        except:
            print("Il job non è ancora completato")
            return None
        return t
    
    @property
    def time(self):
        return self.endTime

class EmptyQueue(Exception):
    pass

class queue:
    '''Classe che implementa una coda semplice'''
    def __init__(self):
        self.__Q = []
    
    def enqueue(self, object):
        self.__Q.insert(0,object)
        
    def dequeue(self):
        if self.is_empty():
            raise EmptyQueue
        return self.__Q.pop()
    
    def is_empty(self):
        return self.__Q == []

class priority_queue:
    '''Implementa una coda con priorità (min-based) di oggetti arbitrari, che
       devono avere la sola proprietà di essere confrontabili con l'operatore <.
       L'attuale implementazione usa una semplice lista non ordinata. Questo
       rende efficiente l'inserzione ma non la ricerca e l'estrazione
       dell'oggetto con minimo valore di priorità'''
       
    def __init__(self, L=[]):
        if not L:
            self.__list = []
        else:
            self.__list = L

    def insert(self, obj):
        self.__list.append(obj)

    def is_empty(self):
        return self.__list == []

    def size(self):
        return len(self.__list)
    
    def minimum(self):
        if self.is_empty():
            raise EmptyQueue
        minindex = 0
        for i in range(1, len(self.__list)):
            if self.__list[i] < self.__list[minindex]:
                minindex = i
        return self.__list[minindex]
        
    def delete(self):
        x = self.minimum()
        self.__list.remove(x)
        return x

class server:
    '''Classe che implementa i server'''
    
    def __init__(self, speed, system, time, num):
        @negexp(speed)
        def JobService():
            return random()
        
        self.time = time
        self.__system = system
        self.__scheduler = JobService
        self.__busy = False
        self.__id = num
        
    def runJob(self, job):
        if not self.__busy:
            self.__busy = True
            self.__currentJob = job
            duration = self.__scheduler()
            job.inService(self.time(), self, duration)
    
    def endJob(self):
        self.__busy = False
        self.__system.jobEnded(self, self.__currentJob)
          
    @property
    def busy(self):
        return self.__busy
    
    @property
    def server(self):
        return self.__id

class system:
    '''Classe che implementa il controllo dei k server e della coda dei job
    in attesa di andare in esecuzione'''
    def __init__(self, k, speed, pQ, time):
        self.__k = k
        self.__servers = [server(speed, self, time, i+1) for i in range(k)]
        self.__Q = queue()
        self.__pQ = pQ
        self.__log = []
        self.time = time
    
    def jobEnters(self, job):
        s = self.available()
        if s:
            s.runJob(job)
            self.__pQ.insert(job)
        else:
            self.__Q.enqueue(job)
    
    def available(self):
        for s in self.__servers:
            if not s.busy:
                return s
        return None

    def jobEnded(self, server, job):
        self.__log.append(job)
        if not self.__Q.is_empty():
            self.jobEnters(self.__Q.dequeue())

    @property
    def log(self):
        return self.__log
    
def sim(numservers=1, speed=1, arrivalRate=1, simLen=2):
    '''Routine di simulazione. Si tratta di un simulatore a eventi discreti.
    il tempo viene fatto avanzare attraverso gli istanti di accadimento
    dei due eventi fondamentali, cioè l'arrivo di una nuova richiesta di
    servizio (job), e il completamento di un job. Il tempo di inizio servizio
    di un job j non è un evento rilevante, perché esso coincide: (1) con il tempo
    di arrivo di j, se almeno un server è disponibile, oppure (2) con il tempo di 
    fine servizio di un job, se j stava aspettando in coda.
    Il tempo viene gestito da un oggetto della classe globalTime  (piuttosto
    che da una semplice variabile globale)
    '''
    typeof = eventType()
    job.resetCounter()
    time = globalTime()
    
    @negexp(arrivalRate)
    def JobArrival():
        return random()
    
    Q = priority_queue()
    S = system(numservers, speed, Q, time)
    Q.insert(arrival(time()))
    
    while time()<simLen:
        e = Q.delete()
        time.advance(e.time)
        if typeof(e) == eventType.JOB_COMPLETION:
            e.server.endJob()
        else:  # typeof(e) == eventType.JOB_ARRIVAL
            j = job(time())
            S.jobEnters(j)
            nextArrival = JobArrival()
            #print(job._job__jobCnt-1, time()+nextArrival)
            Q.insert(arrival(time()+nextArrival))
    
    log = sorted(S.log, key = lambda j: j.arrivalTime)
    
    for j in log:
        print(j)

In [2]:
sim(numservers=1, speed=1, arrivalRate=1, simLen=5)

job 1
	arrival time: 0
	server: 1
	service time: 0
	end time: 1.7796894782504404
job 2
	arrival time: 1.1448952511279955
	server: 1
	service time: 1.7796894782504404
	end time: 2.3503017171045584
job 3
	arrival time: 1.3018581358611956
	server: 1
	service time: 2.3503017171045584
	end time: 5.226607273053581


In [3]:
sim(numservers=2, speed=1, arrivalRate=2, simLen=5)

job 1
	arrival time: 0
	server: 1
	service time: 0
	end time: 0.8048289927414366
job 2
	arrival time: 0.23919701555139075
	server: 2
	service time: 0.23919701555139075
	end time: 1.7725922002114867
job 3
	arrival time: 0.5043917196706661
	server: 1
	service time: 0.8048289927414366
	end time: 1.0144444779014998
job 4
	arrival time: 0.7052062548727772
	server: 1
	service time: 1.0144444779014998
	end time: 1.6749352117844203
job 5
	arrival time: 0.9324236526451409
	server: 1
	service time: 1.6749352117844203
	end time: 1.7355358691740599
job 6
	arrival time: 1.6939925617716207
	server: 1
	service time: 1.7355358691740599
	end time: 1.9332886902759354
job 7
	arrival time: 1.7969828126696477
	server: 2
	service time: 1.7969828126696477
	end time: 2.15049075611093
job 8
	arrival time: 1.8904548344776246
	server: 1
	service time: 1.9332886902759354
	end time: 2.390922917503692
job 9
	arrival time: 2.369752031566729
	server: 2
	service time: 2.369752031566729
	end time: 2.416527887062912
job